<a href="https://colab.research.google.com/github/Gojo-Satoru-git/GEN-AI/blob/main/Resume%20analyzer(uisng%20%20langchain).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pypdf

In [7]:
import json
import re
import os
import openai
from langchain.document_loaders import PyPDFLoader, UnstructuredWordDocumentLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Set OpenRouter API Key
openai.api_key = "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"
openai.api_base = "https://openrouter.ai/api/v1"
print(openai.api_key)
# Use an open-source model from OpenRouter
MODEL_ID = "mistralai/mistral-7b-instruct"  # Change if needed

# Load Open-Source Embeddings for Similarity Matching
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

def extract_text_from_resume(file_path):
    if file_path.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
    elif file_path.endswith(".docx") or file_path.endswith(".doc"):
        loader = UnstructuredWordDocumentLoader(file_path)
    else:
        raise ValueError("Unsupported file format")

    documents = loader.load()
    return " ".join([doc.page_content for doc in documents])

# Fix JSON Parsing by Handling Empty/Invalid Response
def extract_resume_details(text):
    response = openai.ChatCompletion.create(
        model=MODEL_ID,
        messages=[
            {"role": "system", "content": "You are a resume parser. Extract Name, Contact, Skills, Education, Experience, Certifications in JSON format."},
            {"role": "user", "content": text}
        ],
        temperature=0
    )

    raw_response = response["choices"][0]["message"]["content"].strip()

    # Ensure response is valid JSON
    try:
        if raw_response.startswith("{") and raw_response.endswith("}"):
            extracted_info = json.loads(raw_response)
        else:
            extracted_info = json.loads(re.search(r"\{.*\}", raw_response, re.DOTALL).group())
    except (json.JSONDecodeError, AttributeError):
        print("❌ Error: Model did not return valid JSON. Full response below:\n", raw_response)
        return {}

    return extracted_info

def compare_skills(extracted_skills, job_description):
    job_vector = embedding_model.embed_query(job_description)
    skill_vectors = [(skill, embedding_model.embed_query(skill)) for skill in extracted_skills]

    matching_skills = [skill for skill, vec in skill_vectors if sum(a * b for a, b in zip(job_vector, vec)) > 0.5]
    score = (len(matching_skills) / len(extracted_skills)) * 100 if extracted_skills else 0

    return matching_skills, round(score, 2)

def analyze_resume(file_path, job_description):
    text = extract_text_from_resume(file_path)
    extracted_info = extract_resume_details(text)

    if not extracted_info:  # Check if extraction failed
        return json.dumps({"error": "Failed to extract details from resume"}, indent=4)

    matching_skills, fit_score = compare_skills(extracted_info.get("Skills", []), job_description)
    extracted_info["Matching Skills"] = matching_skills
    extracted_info["Fit Score"] = fit_score

    return json.dumps(extracted_info, indent=4)

if __name__ == "__main__":
    file_path = "Resume_Sample.pdf"  # Change to your file path
    job_desc = "We are looking for a Cloth Designer with industrial experience"

    result = analyze_resume(file_path, job_desc)
    print(result)


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
{
    "Name": "John Doe",
    "Contact": {
        "Email": "johndoe@example.com",
        "Phone": "+1 234 567 8901"
    },
    "Location": "123 Fashion St, New York, NY",
    "Summary": "Experienced Cloth Designer with 5+ years in industrial fashion design. Proficient in pattern making, textile selection, and garment construction. Passionate about blending creativity with market trends.",
    "Skills": [
        "Cloth Designing",
        "Textile Selection",
        "Industrial Fashion Trends",
        "Pattern Making",
        "Sewing & Garment Construction",
        "Adobe Illustrator & Photoshop"
    ],
    "Experience": [
        {
            "Title": "Senior Cloth Designer",
            "Company": "XYZ Fashion House",
            "Years": "2019 - Present",
            "Responsibilities": [
                "Designed over 200 unique fashion pieces for industrial clients.",
                "Managed textile 

In [2]:
!pip install openai==0.28

In [9]:
import json
import re
import os
import openai
from langchain.document_loaders import PyPDFLoader, UnstructuredWordDocumentLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Set OpenRouter API Key
openai.api_key = "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"
openai.api_base = "https://openrouter.ai/api/v1"

# Use an open-source model from OpenRouter
MODEL_ID = "mistralai/mistral-7b-instruct"

# Load Open-Source Embeddings for Similarity Matching
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

def extract_text_from_resume(file_path):
    if file_path.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
    elif file_path.endswith(".docx") or file_path.endswith(".doc"):
        loader = UnstructuredWordDocumentLoader(file_path)
    else:
        raise ValueError("Unsupported file format")

    documents = loader.load()
    return " ".join([doc.page_content for doc in documents])

# Fix JSON Parsing by Handling Empty/Invalid Response
def extract_resume_details(text):
    response = openai.ChatCompletion.create(
        model=MODEL_ID,
        messages=[
            {"role": "system", "content": "You are a resume parser. Extract Name, Contact, Skills, Education, Experience, Certifications in JSON format."},
            {"role": "user", "content": text}
        ],
        temperature=0
    )

    raw_response = response["choices"][0]["message"]["content"].strip()

    # Ensure response is valid JSON
    try:
        if raw_response.startswith("{") and raw_response.endswith("}"):
            extracted_info = json.loads(raw_response)
        else:
            extracted_info = json.loads(re.search(r"\{.*\}", raw_response, re.DOTALL).group())
    except (json.JSONDecodeError, AttributeError):
        print("❌ Error: Model did not return valid JSON. Full response below:\n", raw_response)
        return {}

    return extracted_info

def compare_skills(extracted_skills, job_description):
    job_vector = embedding_model.embed_query(job_description)
    skill_vectors = [(skill, embedding_model.embed_query(skill)) for skill in extracted_skills]

    matching_skills = [skill for skill, vec in skill_vectors if sum(a * b for a, b in zip(job_vector, vec)) > 0.5]
    score = (len(matching_skills) / len(extracted_skills)) * 100 if extracted_skills else 0

    return matching_skills, round(score, 2)

def suggest_improvements(extracted_info, job_description):
    suggestions = []

    # Check if key sections are missing
    if not extracted_info.get("Education"):
        suggestions.append("🔹 Add your educational background (e.g., Bachelor’s or Master’s degree).")

    if not extracted_info.get("Experience"):
        suggestions.append("🔹 Include relevant work experience, internships, or projects.")

    if not extracted_info.get("Certifications"):
        suggestions.append("🔹 Add industry-relevant certifications to strengthen your resume.")

    # Suggest soft skills if missing
    essential_soft_skills = ["Communication", "Teamwork", "Problem-solving", "Time Management"]
    missing_soft_skills = [skill for skill in essential_soft_skills if skill not in extracted_info.get("Skills", [])]

    if missing_soft_skills:
        suggestions.append(f"🔹 Consider adding soft skills like {', '.join(missing_soft_skills)}.")

    # Suggest technical skills based on job description
    job_keywords = job_description.lower().split()
    suggested_skills = [word.capitalize() for word in job_keywords if word not in extracted_info.get("Skills", [])]

    if suggested_skills:
        suggestions.append(f"🔹 Improve your technical skills in {', '.join(suggested_skills)}.")

    # Final suggestion if resume score is low
    if extracted_info.get("Fit Score", 100) < 50:
        suggestions.append("🔹 Your resume is not well-aligned with the job. Consider tailoring it to match the job description.")

    return suggestions

def analyze_resume(file_path, job_description):
    text = extract_text_from_resume(file_path)
    extracted_info = extract_resume_details(text)

    if not extracted_info:  # Check if extraction failed
        return json.dumps({"error": "Failed to extract details from resume"}, indent=4)

    matching_skills, fit_score = compare_skills(extracted_info.get("Skills", []), job_description)
    extracted_info["Matching Skills"] = matching_skills
    extracted_info["Fit Score"] = fit_score

    # Generate improvement suggestions
    extracted_info["Suggestions"] = suggest_improvements(extracted_info, job_description)

    return json.dumps(extracted_info, indent=4)

if __name__ == "__main__":
    file_path = "Resume_Sample.pdf"  # Change to your file path
    job_desc = "We are looking for a Cloth Designer with industrial experience"

    result = analyze_resume(file_path, job_desc)
    print(result)


{
    "Name": "John Doe",
    "Contact": {
        "Email": "johndoe@example.com",
        "Phone": "+1 234 567 8901"
    },
    "Location": "123 Fashion St, New York, NY",
    "Summary": "Experienced Cloth Designer with 5+ years in industrial fashion design. Proficient in pattern making, textile selection, and garment construction. Passionate about blending creativity with market trends.",
    "Skills": [
        "Cloth Designing",
        "Textile Selection",
        "Industrial Fashion Trends",
        "Pattern Making",
        "Sewing & Garment Construction",
        "Adobe Illustrator & Photoshop"
    ],
    "Experience": [
        {
            "Title": "Senior Cloth Designer",
            "Company": "XYZ Fashion House",
            "Years": "2019 - Present",
            "Responsibilities": [
                "Designed over 200 unique fashion pieces for industrial clients.",
                "Managed textile sourcing and selection for major clothing brands.",
                "Super